In [1]:
import uniprot
import word2num
import splits

In [2]:
ds = uniprot.Dataset()
release, df = ds.load('sprot')

Workdir=/tmp/tmpce1wmlmc


In [3]:
df.head()

,dataset,name,organism,proteinexistence,sequence
ID,,,,,
Q6GZX4,Swiss-Prot,001R_FRG3G,Frog virus 3 (isolate Goorha),4,MAFSAEDVLKEYDRRRRMEALLLSLYYPNDRKLLDYKEWSPPRVQV...
Q6GZX3,Swiss-Prot,002L_FRG3G,Frog virus 3 (isolate Goorha),4,MSIIGATRLQNDKSDTYSAGPCYAGGCSAFTPRGTCGKDWDLGEQT...
Q197F8,Swiss-Prot,002R_IIV3,Invertebrate iridescent virus 3,4,MASNTVSAQGGSNRPVRDFSNIQDVAQFLLFDPIWNEQPGSIVPWK...
Q197F7,Swiss-Prot,003L_IIV3,Invertebrate iridescent virus 3,4,MYQAINPCPQSWYGSPQLEREIVCKMSGAPHYPNYYPVHPNALGGA...
Q6GZX2,Swiss-Prot,003R_FRG3G,Frog virus 3 (isolate Goorha),3,MARPLLGKTSSVRRRLESLSACSIFFFLRKFCQKMASLVFLNSPVY...


In [4]:
len(df)

561568

In [5]:
df2 = uniprot.transform(df, max_seq_len=1400)
len(df2)

554400

In [6]:
vocab, tok, ids = word2num.transform(df2, lambda seq: [t for t in seq])

554400it [01:20, 6866.21it/s]


In [7]:
ids[:10]

['Q6GZX4',
 'Q6GZX3',
 'Q197F8',
 'Q197F7',
 'Q6GZX2',
 'Q6GZX1',
 'Q197F5',
 'Q6GZX0',
 'Q91G88',
 'Q6GZW9']

In [8]:
index = [x for x in range(len(df2))]
ix_train, ix_valid, ix_test = splits.make_train_validation_test(index, train_size=0.8, validation_size=0.1)


/home/kotliaro/anaconda3/envs/env3.6/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [9]:
print("train={} valid={} test={}".format(len(ix_train), len(ix_valid), len(ix_test)))

train=443520 valid=55440 test=55440


In [10]:
X_train = tok[ix_train]
X_valid = tok[ix_valid]

In [11]:
import train_lm


In [12]:
config = {"embed_size": 400,
         "hidden_units": 1400,
         "num_layers": 3,
         "pad_index": 0,
         "tie_encoder": True}
train_lm.train_lm(vocab, X_train, X_valid, "/tmp", "AWD_LSTM", config)

In [13]:
import fastai
import fastai.text

In [14]:
fastai_vocab = fastai.text.Vocab({index: word for index, word in enumerate(vocab)})

In [18]:
workdir = "/home/kotliaro/tmp"
src = fastai.text.ItemLists(workdir,
                            fastai.text.TextList(items=X_train, 
                                                 vocab=fastai_vocab, 
                                                 path=workdir,
                                                 processor=[]),
                            fastai.text.TextList(items=X_valid,
                                                 vocab=fastai_vocab, 
                                                 path=workdir,
                                                 processor=[]))
src = src.label_for_lm()
data_lm = src.databunch(bs=96, bptt=70)


In [19]:
dir(data_lm)


['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_grab_dataset',
 '_init_ds',
 'add_test',
 'add_tfm',
 'batch_size',
 'create',
 'device',
 'dl',
 'dl_tfms',
 'dls',
 'empty_val',
 'export',
 'fix_dl',
 'fix_ds',
 'from_csv',
 'from_df',
 'from_folder',
 'from_ids',
 'from_tokens',
 'is_empty',
 'label_list',
 'load',
 'load_empty',
 'loss_func',
 'one_batch',
 'one_item',
 'path',
 'remove_tfm',
 'sanity_check',
 'save',
 'show_batch',
 'single_dl',
 'single_ds',
 'test_dl',
 'test_ds',
 'train_dl',
 'train_ds',
 'valid_dl',
 'valid_ds']

In [26]:
data_lm.save()

In [27]:
def make_configuration(arch, config):
    """
    """
    if arch == "AWD_LSTM":
        config_lm = fastai.text.awd_lstm_lm_config.copy()
        config_lm["emb_sz"]      = config["embed_size"]
        config_lm["n_hid"]       = config["hidden_units"]
        config_lm["n_layers"]    = config["num_layers"]
        config_lm["pad_token"]   = config["pad_index"]
        config_lm["tie_weights"] = config["tie_encoder"] # tie embedding and output for LMs
        return config_lm
    raise ValueError("Unknown architecture")

In [ ]:
arch = "AWD_LSTM"
config = {"embed_size": 400,
         "hidden_units": 1150,
         "num_layers": 3,
         "pad_index": 0,
         "tie_encoder": True}


def get_arch(arch):
    if arch == "AWD_LSTM":
        return fastai.text.AWD_LSTM
    else:
        raise ValueError("Unsupported architecture")

        
learner = fastai.text.language_model_learner(data_lm,
                                            get_arch(arch),
                                            config=make_configuration(arch, config),
                                            pretrained=False,
                                            drop_mult=0.5,
                                            clip=25,
                                            wd=1e-7)

In [29]:
learner.lr_find()

LR Finder is complete, type {learner_name}.recorder.plot() to see the graph.


KeyboardInterrupt: 